In [1]:
import syft as sy
import torch as th
import copy
import random

from syft.workers.plan import create_plan

In [2]:
hook = sy.TorchHook(th)
bob = sy.VirtualWorker(hook, id="bob")

In [3]:
def plan_blueprint(data):
    return data.abs()

plan = create_plan(plan_blueprint, th.tensor([-1,-2]))

In [4]:
plan_ptr = sy.local_worker.send(obj=plan, workers=bob)

In [5]:
x = th.tensor([1,2,-4]).send(bob)

In [6]:
response = plan_ptr(x)

In [7]:
response.get()

tensor([1, 2, 4])

In [9]:
bob._objects

{b'plan': <Plan id:b'plan' #tensors:0>, 97039753603: tensor([ 1,  2, -4])}